TODO

1. Conferir perguntas Decision Stump
2. Adicionar Pesos dos pontos

In [ ]:
import os, time

import numpy as np
import matplotlib as plt
import pandas as pd

pd.options.display.max_rows = 10

In [ ]:
data = pd.read_csv('data/tic-tac-toe.data', header=None)
data

In [ ]:
# Armazenar o label correto dos numeros
data = data.rename(columns = {9:'label'})
data['label'] = data['label'].replace({'positive': 1, 'negative': -1})

# Separar 20% para teste
train = data.sample(frac = 0.8)
test = data.drop(train.index)

y_train = train.label
y_test  = test.label

# Retirar a primeira coluna referente aos labels
X_train = train.drop("label", axis=1)
X_test  = test.drop("label", axis=1)

In [ ]:
class DecisionStump:
    def __init__(self):
        # Possiveis categorias que o dado vai ter para poder realizar as perguntas (Tem x na posição 0?)
        # No caso do tick-tack-toe vai ser x, o e b
        self.unique_categories = []
        # Dimensionalidade do dado, para saber quantas possíveis posições pode ter
        self.data_dimension = 0
        
        self.question_id  = None
        self.question_cat = None
        
    def fit(self, X, y, weights=None):
        """Seleciona o modelo com base em X, y e os pesos de cada entrada"""
        
        # Guarda todos os valores possiveis para todas as colunas e a dimensao dos dados
        # No caso do Tick-Tack-Toe, sera x,o,b para todas
        for column in X:
            self.unique_categories.append(X[column].unique())
        self.data_dimension = len(list(X))
        
        best_model_error  = 2
        for i in range(0, self.data_dimension):
            for cat in self.unique_categories[0]:
                error = self._compute_training_error(X, y, i, cat)
                if (error < best_model_error):
                    best_model_error  = error
                    self.question_id  = i
                    self.question_cat = cat
                    print('Best model {} and {} with error {}'.format(self.question_id, self.question_cat, error))
        
        print('Best model {} and {}'.format(self.question_id, self.question_cat))
        
        return
    
    def predict(X):
        """Prevê os valores das entradas em X"""
        if (self.question_id == None and self.question_cat == None):
            print('Model was not fitted')
            return
        
        return
    
    def _compute_training_error(self, X, y, question_id, question_cat, weights=None):
        nb_samples  = X.count()[0]
        right_count = 0
        for i, row in X.iterrows():
            if ( (row[question_id] == question_cat and y[i] == 1) or
                    (row[question_id] != question_cat and [i] != -1)):
                right_count += 1
        
        return right_count / nb_samples 
        
    
dt = DecisionStump()
dt.fit(X_train, y_train)